# Export Data

In [70]:
from packages import *
%matplotlib inline

In [71]:
from collections import Counter

In [72]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import vgg16

In [73]:
subject = 1
data_session = 2
data_version = ['c', 'd']
label_version = 'c'

In [74]:
sample_data = True
save_data = True

In [75]:
X = {}
for v in data_version:
    X[v] = numpy_load('X_sub-{}_ses-{}_{}.npy'.format(subject, data_session, v), os.path.join(DATASET_X_DIR, v))
Y = numpy_load('Y_sub-{}_ses-{}_{}.npy'.format(subject, data_session, label_version), os.path.join(DATASET_Y_DIR, label_version))

In [76]:
def convert_bytes(num):
    """
    this function will convert bytes to MB.... GB... etc
    """
    step_unit = 1000.0 #1024 bad the size

    for x in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if num < step_unit:
            return "%3.1f %s" % (num, x)
        num /= step_unit

In [77]:
sample_num = 50

In [78]:
if sample_data:
    indices = np.arange(len(Y))
    sample_indices = np.random.choice(indices, sample_num, replace=False)

In [79]:
if sample_data:
    X_sample = {}
    for key in X:
        X_sample[key] = np.take(X[key], sample_indices, axis=0)
    Y_sample = np.take(Y, sample_indices, axis=0)
else:
    # read from .gz
    pass

In [80]:
print 'Data size: {}'.format(len(Y))
for key in X:
    print 'X[{}] size: {}'.format(key, convert_bytes(X[key].nbytes))
print 'Y size: {}'.format(convert_bytes(Y.nbytes))

Data size: 370
X[c] size: 148.9 MB
X[d] size: 18.0 MB
Y size: 11.1 MB


In [81]:
print 'Sample size: {}'.format(len(Y_sample))
for key in X:
    print 'X_sample[{}] size: {} bytes'.format(key, convert_bytes(X_sample[key].nbytes))
print 'Y_sample size: {} bytes'.format(convert_bytes(Y_sample.nbytes))

Sample size: 50
X_sample[c] size: 20.1 MB bytes
X_sample[d] size: 2.4 MB bytes
Y_sample size: 1.5 MB bytes


In [83]:
if save_data:
    submission_data_dir = os.path.join(DUMP_DIR, 'submission/')
    for key in data_version:
        numpy_save(X[key], 'test-data_X_sub-{}_ses-{}_{}'.format(subject, data_session, key), submission_data_dir)
    numpy_save(Y, 'test-data_Y_sub-{}_ses-{}_{}'.format(subject, data_session, label_version), submission_data_dir)
    print 'Saved data samples.'

Saved data samples.


## Gzip data

In [52]:
import gzip

In [84]:
def gzip_file(data, filename, dir):
    filename = os.path.join(dir, '{}.gz'.format(filename))
    with gzip.open(filename, 'wb') as f:
        numpy_save(data, f)
    print 'Wrote to {}. Size: {}'.format(filename, convert_bytes(os.path.getsize(filename)))

In [87]:
data_version[0], data_version[1]

('c', 'd')

In [114]:
submission_data_dir = os.path.join(DUMP_DIR, 'submission/')
#     for key in data_version:
#         gzip_file(X_sample[key], 'test-data_X_sub-{}_ses-{}_{}'.format(subject, data_session, key), submission_data_dir)
#     gzip_file(Y_sample, 'test-data_Y_sub-{}_ses-{}_{}'.format(subject, data_session, label_version), submission_data_dir)
#     print 'Saved gzipped data samples.'
# else:
#     print 'Not saving gzipped data.'

# data = {}
X_bold = X_sample[data_version[0]]
X_voxel = X_sample[data_version[1]]
Y = Y_sample

if save_data:
    name = os.path.join(submission_data_dir, 'test-data_sub-{}_ses-{}'.format(subject, data_session))
    
    np.savez_compressed(name, x_bold=X_bold, x_voxel=X_voxel, y=Y)
    print 'Saved zipped data samples.'
else:
    print 'Not saving zipped data.'


Saved zipped data samples.


In [115]:
loaded = np.load('{}.npz'.format(name))

In [116]:
loaded.keys()

['y', 'x_voxel', 'x_bold']

In [117]:
for k in loaded:
    print k, loaded[k].shape

y (50, 7500)
x_voxel (50, 7500)
x_bold (50, 52, 43, 45)


# Export model

In [55]:
import yaml

In [56]:
subject = 1
session = 1
model_version = 'n-no-reg'

In [57]:
model_name = 'sub-{}_ses-{}_mod-{}_data-{}_lab-{}'.format(subject, session, model_version, '-'.join(data_version), label_version)

In [90]:
loading_model = False
saving_model = False

In [91]:
from tensorflow.keras.models import load_model

In [92]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [93]:
if loading_model:
    test_model = load_model(os.path.join(TRAINED_MODELS_DIR, model_name), compile=False)
else:
    print 'Not loading model.'

Not loading model.


In [94]:
if saving_model:
    yaml_string = test_model.to_yaml()
    architecture_filename = '{}_architecture.yml'.format(model_name)
    print architecture_filename
    with open(architecture_filename, 'wb') as f:
        yaml.dump(yaml_string, f)